In [1]:
print("Ok")

Ok


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
# from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\Mariam\anaconda3\envs\metachatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY="613f5901-3ea4-4b1b-9f65-f6b4d8556c34"
PINECONE_API_ENV="aws-starter" #Serverless
# PINECONE_API_ENV="gcp-starter"

### Load Data

In [3]:
# Extract data from the Pdf
def load_pdf(data):
    loader= DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyMuPDFLoader)
    documents= loader.load()
    return documents

In [4]:
extracted_data= load_pdf("data/")

### Create Chuncks

In [5]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_chunks= text_split(extracted_data)

In [7]:
print("length of our chunks", len(text_chunks))

length of our chunks 5779


In [8]:
text_chunks[20]

Document(page_content='advisory board, made up of public librarians, medical\nlibrarians and consumer health experts, evaluated the top-\nics and made suggestions for inclusion. The list was sorted\nby category and sent to GEM2 medical advisors, certified\nphysicians with various medical specialities, for review.\nFinal selection of topics to include was made by the med-\nical advisors in conjunction with the Gale Group editor.\nABOUT THE CONTRIBUTORS\nThe essays were compiled by experienced medical', metadata={'source': 'data\\Medical_book.pdf', 'file_path': 'data\\Medical_book.pdf', 'page': 6, 'total_pages': 637, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creationDate': "D:20041218170002-05'00'", 'modDate': "D:20041218161531-06'00'", 'trapped': ''})

### Download Embedding model

In [9]:
## Download Embedding model from Hugging Face

def download_hugging_face_embedding():
    embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-miniLM-L6-v2")
    return embeddings

In [10]:
embedding_model=download_hugging_face_embedding()

In [11]:
query_result= embedding_model.embed_query("Hello World")
print("Length of query: ", len(query_result))

Length of query:  384


### Store Embedding to VectorDB

In [12]:
from pinecone import Pinecone
#Initializing the Pinecone
pc = Pinecone(api_key="613f5901-3ea4-4b1b-9f65-f6b4d8556c34")
# pinecone.init(api_key=PINECONE_API_KEY)

# pc = Pinecone(api_key="********-****-****-****-************")
# index = pc.Index("quickstart")
index_name= "medicalchat"

# Creating embeddings for each text chunks and storing
docsearch=pc.from_texts([t.page_content for t in text_chunks], embedding_model, index_name=index_name)

AttributeError: from_texts is not a top-level attribute of the Pinecone class provided by pinecone's official python package developed at https://github.com/pinecone-io/pinecone-python-client. You may have a name collision with an export from another dependency in your project that wraps Pinecone functionality and exports a similarly named class. Please refer to the following knowledge base article for more information: https://docs.pinecone.io/troubleshooting/pinecone-attribute-errors-with-langchain


In [13]:
# Connect to the index
index = pinecone.Index(index_name, host="https://medicalchat-8v1hipi.svc.aped-4627-b74a.pinecone.io")

In [14]:
embeddings = [embedding_model.embed_documents(t.page_content) for t in text_chunks]

# Store embeddings in Pinecone
for i, embedding in enumerate(embeddings):
    index.upsert([(str(i), embedding)])

print("Embeddings have been stored in the index.")

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 276480 bytes.

In [13]:
# If we already have an index we can load it like this
host="https://medicalchat-8v1hipi.svc.aped-4627-b74a.pinecone.io"
# If we already have an index we can load it like this
docsearch = pinecone.Index(index_name, host=host)

# Query for similarity search
query = "What are Allergies"

# Generate the embedding for the query
query_embedding = embedding_model.embed_documents([query])[0]

# Perform similarity search
result = docsearch.query(queries=[query_embedding], top_k=3)

# Extract and print the results
docs = [match['id'] for match in result['matches']]
print("Result", docs)

NameError: name 'index_name' is not defined

In [15]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [17]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])